# Modelo de Mercado (Unifactorial)


El modelo de de mercado unifactorial consiste en tratar de explicar la rentabilidad de un activo a través del comportamiento del mercado, medido a través de la evolución de un indice. Es decir, se trata de realizar una regresión lineal unifactorial entre el comportamiento del indice y el comportamiento del mercado.

$$\tilde{R}_{it}=\alpha_i+\beta_i \tilde{R}_{Mt}+\tilde{\varepsilon}_{it}$$

La rentabilidad depende de tres componentes:
- $\alpha_i$: la parte del riesgo explicada por determinadas características del riesgo que no se deben a la evolución de la economía sino a características predecibles del título, se supone constante en el tiempo.
- $\beta_i$:la parte del riesgo que obedece a la evolución general de la economía que vienen recogido por el índice de mercado (RIESGO SISTEMÁTICO)
- $\varepsilon_{it}$:perturbación aleatoria del título, provocado por las decisiones o acontecimientos no predecibles (RIESGO ESPECÍFICO)

## Hipótesis del modelo

- El riesgo específico es una variable aleatoria con $E(\tilde{\varepsilon}_{it})=0$. Esto tiene sentido,cualquir componente predecible debe estar incluido en $\alpha_i$.
- El riesgo específico y el sistemático deben estar incorrelacionados $cov(R_{Mt},\varepsilon_{it}), \forall t$.
- No debe existir correlación entre los riesgos específicos de dos activos distintos $cov(\tilde{\varepsilon}_{it},\tilde{\varepsilon}_{jt}), \forall t$.
- No debe existir correlación entre los errores a lo largo del tiempo  $cov(\tilde{\varepsilon}_{it},\tilde{\varepsilon}_{jt'}),\forall t\neq t'$
- La varianza debe del riesgo específico ser constante a lo largo del tiempo $\sigma^2(\tilde{\varepsilon}_{it})=\sigma^2(\tilde{\varepsilon}_{it'}),\forall t\neq t'$.
- El riesgo sistemático se comporta como $\tilde{\varepsilon}_{it}=N(0,1),\forall t$

In [191]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

In [278]:
RISKY_ASSET = 'AMZN'
MARKET_BENCHMARK = '^GSPC'
START_DATE = '2014-01-01'
END_DATE = '2018-12-31'

In [279]:
df = yf.download([RISKY_ASSET, MARKET_BENCHMARK],
                 start=START_DATE,
                 end=END_DATE,)['Adj Close']
df.head()

[*********************100%***********************]  2 of 2 completed


,AMZN,^GSPC
Date,,
2014-01-02,19.898500,1831.979980
2014-01-03,19.822001,1831.369995
2014-01-06,19.681499,1826.770020
2014-01-07,19.901501,1837.880005
2014-01-08,20.096001,1837.489990


Tenemos los datos diarios y queremos trabajar con datos mensuales. Para ello generamos un nuevo dataframe `X` y utilizamos la función `resample('M').last()` tomando como referencia el último día del mes con cotización.

In [280]:
df.index = pd.to_datetime(df.index)
X=df.resample('M').last()
X.head()

,AMZN,^GSPC
Date,,
2014-01-31,17.934500,1782.589966
2014-02-28,18.105000,1859.449951
2014-03-31,16.818501,1872.339966
2014-04-30,15.206500,1883.949951
2014-05-31,15.627500,1923.569946


In [281]:
monthly_ret = np.log(X).diff().dropna()
monthly_ret.head()

,AMZN,^GSPC
Date,,
2014-02-28,0.009462,0.042213
2014-03-31,-0.073709,0.006908
2014-04-30,-0.100757,0.006182
2014-05-31,0.027309,0.020812
2014-06-30,0.038384,0.018879


Calculamos la $\beta$ utilizando las varianza:
$$\beta=\frac{cov(AMZN,GSPC)}{\sigma^2(GSPC)}$$

In [282]:
covariance = monthly_ret.cov().iloc[0,1]
benchmark_variance = monthly_ret.cov().iloc[1,1]
beta = covariance / benchmark_variance
beta

1.6106258640720965

Estimamos la beta utilizando una regresión lineal para ello:
1. `y = monthly_ret.pop('AMZN')`: extraemos los datos de `AMZN`en una serie que denominamos `y`.
2. `monthly_ret=sm.add_constant(monthly_ret)`:añadimos una fila de unos en el dataframe `monthly_ret`.
3. `capm_model = sm.OLS(y, monthly_ret).fit()`: realizamos la regresión por mínimos cuadrados ordinarios.

In [283]:
y = monthly_ret.pop('AMZN')
y.head()

Date
2014-02-28    0.009462
2014-03-31   -0.073709
2014-04-30   -0.100757
2014-05-31    0.027309
2014-06-30    0.038384
Freq: M, Name: AMZN, dtype: float64

In [284]:
type(y)

pandas.core.series.Series

In [285]:
monthly_ret=sm.add_constant(monthly_ret)
monthly_ret.head()

,const,^GSPC
Date,,
2014-02-28,1.0,0.042213
2014-03-31,1.0,0.006908
2014-04-30,1.0,0.006182
2014-05-31,1.0,0.020812
2014-06-30,1.0,0.018879


In [249]:
capm_model = sm.OLS(y, monthly_ret).fit()
print(capm_model.summary())

                            OLS Regression Results                            
Dep. Variable:                   AMZN   R-squared:                       0.379
Model:                            OLS   Adj. R-squared:                  0.368
Method:                 Least Squares   F-statistic:                     34.78
Date:                Mon, 06 Nov 2023   Prob (F-statistic):           2.11e-07
Time:                        11:04:18   Log-Likelihood:                 77.379
No. Observations:                  59   AIC:                            -150.8
Df Residuals:                      57   BIC:                            -146.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0149      0.009      1.701      0.0

## Consideramos el activo libre de riesgo

Una estrategia alternativa para estimar la beta es incorporar en la estimación el activo libre de riesgo. Si:
- $\tilde{\pi}_{it}=\tilde{R}_{it}-R_f$
- $\tilde{\pi}_{Mt}=\tilde{R}_{Mt}-R_f$

Por tanto planteamos el siguiente modelo. 

$$\tilde{\pi}_{it}=\alpha_i+\beta_i \tilde{pi}_{Mt}+\tilde{\varepsilon}_{it}$$

Utilizando datos del prof. Kenneth French: La prima de mercado $R_m-R_f$ y el tipo sin riesgo (aproximado por la letra del Tesoro a un mes) pueden descargarse del [sitio web](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html) del profesor Kenneth French . Tenga en cuenta que la definición de referencia de mercado utilizada por el prof. French es diferente a la del índice S&P 500. En su sitio web encontrará una descripción detallada. 

Veamos como podemos trabajar con dichos datos:

1. Descargamos los datos: `!wget http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip`
2. Descomprimimos el archivo: `!unzip -a F-F_Research_Data_Factors_CSV.zip`. La opción `-a` asegura que los archivos de texto extraídos tengan el formato de fin de línea correcto para el sistema operativo.
3. Eliminados el archivo zip: `!rm F-F_Research_Data_Factors_CSV.zip`.



In [114]:
!wget http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip
!unzip -a F-F_Research_Data_Factors_CSV.zip
!rm F-F_Research_Data_Factors_CSV.zip

--2023-11-06 10:14:02--  http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip
Resolving mba.tuck.dartmouth.edu (mba.tuck.dartmouth.edu)... 129.170.136.60
Connecting to mba.tuck.dartmouth.edu (mba.tuck.dartmouth.edu)|129.170.136.60|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12773 (12K) [application/x-zip-compressed]
Saving to: 'F-F_Research_Data_Factors_CSV.zip'

F-F_Research_Data_F 100%[===================>]  12.47K  --.-KB/s    in 0s      

2023-11-06 10:14:03 (196 MB/s) - 'F-F_Research_Data_Factors_CSV.zip' saved [12773/12773]

Archive:  F-F_Research_Data_Factors_CSV.zip
replace F-F_Research_Data_Factors.CSV? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


Recuperamos el archivo para eso utilizamos la función de pandas `read_csv`. Las tres primeras filas tienen información que no nos interesa, por tanto las eliminamos con `skiprows=3`

In [250]:
factor_df = pd.read_csv('F-F_Research_Data_Factors.csv', skiprows=3)

In [251]:
factor_df.head()

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.56,-2.43,0.22
1,192608,2.64,-1.17,3.82,0.25
2,192609,0.36,-1.40,0.13,0.23
3,192610,-3.24,-0.09,0.70,0.32
4,192611,2.53,-0.10,-0.51,0.31


Renombramos las columnas

In [252]:
factor_df.columns = ['date', 'mkt', 'smb', 'hml', 'rf']
factor_df.head()

,date,mkt,smb,hml,rf
0,192607,2.96,-2.56,-2.43,0.22
1,192608,2.64,-1.17,3.82,0.25
2,192609,0.36,-1.40,0.13,0.23
3,192610,-3.24,-0.09,0.70,0.32
4,192611,2.53,-0.10,-0.51,0.31


Nos aseguramos que la información contenida en la columna `date`sea una variable de tipo string. Es decir, una cadena alfanumérica.

In [253]:
factor_df['date'] = factor_df['date'].astype(str)
factor_df.head()

,date,mkt,smb,hml,rf
0,192607,2.96,-2.56,-2.43,0.22
1,192608,2.64,-1.17,3.82,0.25
2,192609,0.36,-1.40,0.13,0.23
3,192610,-3.24,-0.09,0.70,0.32
4,192611,2.53,-0.10,-0.51,0.31


Dado que el formato de la fecha es AAAAMM, eliminamos todas las filas que tengan alguna fecha de más de seis dígitos. 

In [254]:
factor_df = factor_df[factor_df['date'].str.len() == 6]
factor_df.head()

,date,mkt,smb,hml,rf
0,192607,2.96,-2.56,-2.43,0.22
1,192608,2.64,-1.17,3.82,0.25
2,192609,0.36,-1.40,0.13,0.23
3,192610,-3.24,-0.09,0.70,0.32
4,192611,2.53,-0.10,-0.51,0.31


Convertimos la variable `date`en una fecha para ello:
- Indicamos el formato de partida `format='%Y%m'`.
- El parámetro `errors='coerce'` para que, en caso de que haya alguna fecha que aún no coincida con el formato, en lugar de lanzar un error, pandas convierta esos valores en NaT (Not a Time), que es el equivalente de NaN para los datos de tiempo en pandas.
- Sólo debe mostrar año y mes `dt.strftime("%Y-%m")`

In [255]:
factor_df['date'] = pd.to_datetime(factor_df['date'],format='%Y%m', errors='coerce').dt.strftime("%Y-%m")
factor_df.head()                     

,date,mkt,smb,hml,rf
0,1926-07,2.96,-2.56,-2.43,0.22
1,1926-08,2.64,-1.17,3.82,0.25
2,1926-09,0.36,-1.40,0.13,0.23
3,1926-10,-3.24,-0.09,0.70,0.32
4,1926-11,2.53,-0.10,-0.51,0.31


In [256]:
factor_df = factor_df.set_index('date')
factor_df.head()

,mkt,smb,hml,rf
date,,,,
1926-07,2.96,-2.56,-2.43,0.22
1926-08,2.64,-1.17,3.82,0.25
1926-09,0.36,-1.40,0.13,0.23
1926-10,-3.24,-0.09,0.70,0.32
1926-11,2.53,-0.10,-0.51,0.31


Seleccionamos los datos comprendidos entre `START_DATE='2014-01'`y `END_DATE='2018-12'`. Para ello, utilizamos el  código `factor_df = factor_df.loc[START_DATE:END_DATE]`, que utiliza el método `.loc[]`  para filtrar las filas basándose en un rango de fechas.

Posteriormente tendremos que extraer la información del activo libre de riesgo y tenemos que asgurarnos que el index es un fecha (`factor_df.index = pd.to_datetime(factor_df.index)`) y que está en formato año y mes `factor_df.index = factor_df.index.to_period('M')`.

In [266]:
START_DATE='2014-02'
END_DATE='2018-12'
factor_df = factor_df.loc[START_DATE:END_DATE]
factor_df.index = pd.to_datetime(factor_df.index)
factor_df.index = factor_df.index.to_period('M')
factor_df.head()

,mkt,smb,hml,rf
date,,,,
2014-02,0.0465,0.0035,-0.0031,0.0
2014-03,0.0043,-0.0181,0.0493,0.0
2014-04,-0.0019,-0.0419,0.0117,0.0
2014-05,0.0206,-0.0188,-0.0013,0.0
2014-06,0.0261,0.0309,-0.0070,0.0


Dividimos por 100 todos los valores para no trabajar con porcentajes.

In [267]:
factor_df = factor_df.apply(pd.to_numeric, errors='coerce').div(100)
factor_df.head()

,mkt,smb,hml,rf
date,,,,
2014-02,0.000465,0.000035,-0.000031,0.0
2014-03,0.000043,-0.000181,0.000493,0.0
2014-04,-0.000019,-0.000419,0.000117,0.0
2014-05,0.000206,-0.000188,-0.000013,0.0
2014-06,0.000261,0.000309,-0.000070,0.0


In [268]:
y_df = pd.DataFrame(y)
y_df.index = y.index.to_period('M')
y_df.head()

,AMZN
Date,
2014-02,0.009462
2014-03,-0.073709
2014-04,-0.100757
2014-05,0.027309
2014-06,0.038384


In [269]:
y_df.index = y.index.to_period('M')
y_df['rf'] = factor_df['rf']
y_df.head()

,AMZN,rf
Date,,
2014-02,0.009462,0.0
2014-03,-0.073709,0.0
2014-04,-0.100757,0.0
2014-05,0.027309,0.0
2014-06,0.038384,0.0


In [270]:
excess_returns =y_df['AMZN']-y_df['rf']
excess_returns.head()

Date
2014-02    0.009462
2014-03   -0.073709
2014-04   -0.100757
2014-05    0.027309
2014-06    0.038384
Freq: M, dtype: float64

In [286]:
monthly_ret.index = pd.to_datetime(monthly_ret.index)
monthly_ret.index = monthly_ret.index.to_period('M')
monthly_ret['rf'] = factor_df['rf']
monthly_ret.head()

,const,^GSPC,rf
Date,,,
2014-02,1.0,0.042213,0.0
2014-03,1.0,0.006908,0.0
2014-04,1.0,0.006182,0.0
2014-05,1.0,0.020812,0.0
2014-06,1.0,0.018879,0.0


In [287]:
monthly_ret['mkt'] = monthly_ret['^GSPC']-monthly_ret['rf']


In [289]:
monthly_ret = monthly_ret.drop(['^GSPC', 'rf'], axis=1)
monthly_ret.head()

,const,mkt
Date,,
2014-02,1.0,0.042213
2014-03,1.0,0.006908
2014-04,1.0,0.006182
2014-05,1.0,0.020812
2014-06,1.0,0.018879


In [290]:
capm_model = sm.OLS(excess_returns, monthly_ret).fit()
print(capm_model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.379
Model:                            OLS   Adj. R-squared:                  0.368
Method:                 Least Squares   F-statistic:                     34.78
Date:                Mon, 06 Nov 2023   Prob (F-statistic):           2.11e-07
Time:                        11:29:02   Log-Likelihood:                 77.379
No. Observations:                  59   AIC:                            -150.8
Df Residuals:                      57   BIC:                            -146.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0149      0.009      1.702      0.0

## Interpretación del modelo

El riesgo total de un activo se puede descomponer en el riesgo sistemático y el específico:
$$\sigma^2_i=\beta_i\sigma^2_M+ \sigma^2_{\varepsilon_i}$$
Por tanto cuando diversiifcamos lo que estamos eliminando es el riesgo específico $\sigma^2_{\varepsilon_i}$.

La covarianza entre dos activos será:

$$Cov(\tilde{R}_i, \tilde{R}_j)=\beta_i\beta_j\sigma^2_M$$

La correlación entre dos activods será:

$$\rho_{ij}=\frac{\beta_i\beta_j\sigma^2_M}{\sigma_i\sigma_j}=\frac{(\beta_i\sigma^2_M)·(\beta_j\sigma^2_M)}{(\sigma_i\sigma_M)·(\sigma_j\\sigma_M}=\rho_{iM}·\rho_{jM}$$

### Simplificamos significativamente la estimación del CAPM

Si disponemos de n activos tendremos que estimar:
- $2n$ parámetros correspondientes a $\alpha_i$ y $\beta_i$.
- La rentabilidad esperada del mercado $\mu_M$ o su premio $\pi_M$ y su varianza $\sigma^2$.
- Las $n$ medidas del riesgo específico $\sigma^2_{\varepsilon_i}$.

Por tanto necesito $3n+2$ parámetros frente a los $2n+\frac{n(n-1)}{2}$ de la estimación de la matriz de varianzas covarianzas propuesta en el modelo de equilibrio. En el caso de $n=500$ estamos hablando de 1.502 parámetros frente a los 125.250 parámetros.

## La aplicación del modelo de mercado a una cartera

La rentabilidad de una cartera vendrá definida por:

$$\tilde{R}_p=\sum_i^n w_i(\alpha_i+\beta_i\tilde{R}_M+\tilde{\varepsilon}_i)=\sum_i^n w_i \alpha_i+\tilde{R}_M\sum_i^n w_i \beta_i+ \sum_i^n w_i \tilde{\varepsilon}_i=\alpha_p+\beta_p\tilde{R}_M+\tilde{\varepsilon}_p$$

donde:
$\alpha_p=\sum_i^n w_i \alpha_i$, $\beta_p =\sum_i^n w_i \beta_i$, $\tilde{\varepsilon}_p=\sum_i^n w_i \tilde{\varepsilon}_i$

Lo podemos expresar en función del premio por el riesgo:

$$\tilde{\pi}_p=\alpha_p+\beta_p\tilde{\pi}_M+\tilde{\varepsilon}_p$$

La varianza de una cartera:
$$\sigma_p^2=\beta_p^2\sigma^2_M+\sum_{i=1}^n w_i \sigma^2_{\varepsilon_i}=\beta_p^2\sigma^2_M+\sigma^2_{\varepsilon_p}$$

El ratio de Sharpe:
$$\frac{\pi_p}{\sigma_p}=\frac{\sum_i^n w_i \alpha_i+\pi_M\sum_i^n w_i \beta_i}{\left[\sigma^2_M\left(sum_i^n w_i \beta_i\right)^2+\sum_i^n w_i \sigma^2_{\varepsilon_i}\right]^{1/2}}$$